<a href="https://colab.research.google.com/github/YeweiYuan/Quantum-Pricing/blob/main/qpanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

量子期权定价
---



---
蒙特卡罗方法和blm



In [ ]:
import numpy as np
from scipy.stats import norm
from numpy.random import standard_normal

In [ ]:
class Sampling:
  def __init__(self,N=10000,T=0.5,S_0=5.29,K=6,Rate=0.04,Sigma=0.24): #初始化六个月的股票期权，标的资产股票价格为5.29元，期权执行价格为6元，年化无风险利率和年化波动率分别为4%和24%，模拟10000次
    self.N = N # sample size
    self.T = T # time
    self.S_0 = S_0 # init price
    self.Rate = Rate # rate
    self.Sigma = Sigma # volatility
    self.K = K # strike price

  def callMonteCarlo(self): #蒙特卡罗法，看涨期权
    z = standard_normal(self.N)
    S_t = self.S_0*np.exp((self.Rate-0.5*self.Sigma**2)*self.T+self.Sigma*z*np.sqrt(self.T))
    call = np.maximum(0,S_t-self.K)
    res = sum(call)*(np.exp(-self.Rate*self.T))/self.N
    return res
  
  def callBLM(self): #BSM模型，看涨期权
    d1 = (np.log(self.S_0/self.K)+(self.Rate*self.T))/(self.Sigma*np.sqrt(self.T))+0.5*self.Sigma*np.sqrt(self.T)
    d2 = d1-self.Sigma*np.sqrt(self.T)
    call = self.S_0*norm.cdf(d1,0.0,1.0)-self.K*np.exp(-self.Rate*self.T)*norm.cdf(d2,0.0,1.0)
    return call

In [ ]:
test = Sampling()

In [ ]:
print('MonteCarlo: %f \nBLM: %f'%(test.callMonteCarlo(),test.callBLM()))

MonteCarlo: 0.159636 
BLM: 0.153154


---
量子方法

In [ ]:
!pip install pyqpanda

     |████████████████████████████████| 20.8MB 1.3MB/s 


In [ ]:
from pyqpanda import *

In [ ]:
import numpy as np
from scipy.stats import norm
from numpy.random import standard_normal

In [ ]:
class Qsampling:
  def _init_(self,N=15,T=0.5,S_0=5.29,K=6,Rate=0.04,Sigma=0.24):
    self.N = N # qubit size
    self.T = T # time
    self.S_0 = S_0 # init price
    self.Rate = Rate # rate
    self.Sigma = Sigma # volatility
    self.K = K # strike price

  def sample(self):
    x_max = np.sqrt(self.T)*3 #这里取3倍标准差涵盖99.7的分布
    delta = 2*x_max/(2^self.N-1)
    x = []
    for j in range(2^self.N):
      x.append(norm.pdf(-x_max+j*delta))
    x = x/sum(x)

    init(QMachineType.CPU)
    qubits = qAlloc_many(self.N)
    cbits = cAlloc_many(self.N)
    prog = QProg()
    

{'0001': 268, '0111': 249, '1001': 215, '1111': 268}
